In [1]:
import math, time, random, datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-whitegrid')
%matplotlib inline

In [18]:
# Loading data

import sys
sys.path.append('../data')
from dataset_pointers import graph_nodes, graph_edges

nodes = pd.read_csv(graph_nodes, low_memory=False)
edges = pd.read_csv(graph_edges, low_memory=False, index_col='edge_id')
no_nan = pd.read_csv('../data/raw/no_nan_data.csv', low_memory=False, index_col='node_id')

In [ ]:
clean_sample = no_nan.sample(n=10000)

In [19]:
clean = clean_sample.drop(['Account ID String', 'Address', 'Name'], axis=1)

In [24]:
clean.head()

,Label,Revenue Size Flag,Person or Organisation,Income Size Flag,CoreCaseGraphID,ExtendedCaseGraphID,testingFlag
node_id,,,,,,,
1502000,1,4,-1,-1,0,0,NaN
1502001,1,2,-1,-1,0,0,NaN
1502002,1,2,-1,-1,2492,0,0.0
1502003,1,4,-1,-1,0,0,NaN
1502004,1,2,-1,-1,0,0,NaN


In [25]:
clean[clean.isnull().any(1)]

,Label,Revenue Size Flag,Person or Organisation,Income Size Flag,CoreCaseGraphID,ExtendedCaseGraphID,testingFlag
node_id,,,,,,,
1502000,1,4,-1,-1,0,0,NaN
1502001,1,2,-1,-1,0,0,NaN
1502003,1,4,-1,-1,0,0,NaN
1502004,1,2,-1,-1,0,0,NaN
1502007,1,5,-1,-1,0,0,NaN
...,...,...,...,...,...,...,...
3001177741,5,-1,1,-1,0,0,NaN
3001177742,5,-1,2,-1,0,0,NaN
3001177743,5,-1,1,-1,0,0,NaN


In [26]:
clean = clean.drop('testingFlag', axis=1)

In [33]:
clean.head()

,Label,Revenue Size Flag,Person or Organisation,Income Size Flag,CoreCaseGraphID,ExtendedCaseGraphID
node_id,,,,,,
1502000,1,4,-1,-1,0,0
1502001,1,2,-1,-1,0,0
1502002,1,2,-1,-1,2492,0
1502003,1,4,-1,-1,0,0
1502004,1,2,-1,-1,0,0


In [36]:
clean.ExtendedCaseGraphID.value_counts()[:4]

0      286938
333        79
421        68
289        64
Name: ExtendedCaseGraphID, dtype: int64

In [37]:
clean.CoreCaseGraphID.value_counts()[:4]

0       308711
1658         4
3963         4
2170         4
Name: CoreCaseGraphID, dtype: int64

In [31]:
# CLASSES ARE EXTREMELY UNBALANCED -> EXPECT SHITTY RESULT
# 300'000 nodes with case=0 ...